In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

## Zadanie domowe – Detekcja obiektów z wykorzystaniem wzorca probabilistycznego

In [12]:
import os

os.mkdir('roi')

In [13]:
i_pedestrian_counter: int = 0


def process_frame(current_frame: np.ndarray) -> np.ndarray:
    original_image: np.ndarray = current_frame.copy()

    gray_image: np.ndarray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    _, threshold_image = cv2.threshold(gray_image, 45, 255, cv2.THRESH_BINARY)
    median: np.ndarray = cv2.medianBlur(threshold_image, 7)
    # Ta 'heurystyka', to po prostu dylatacja obrazu
    dilated = cv2.dilate(median, np.ones((5, 5), np.uint8), iterations=2)
    closed: np.ndarray = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8), iterations=2)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closed)

    objects_on_screen: bool = num_labels > 1
    if objects_on_screen:
        for object_index in range(1, num_labels):
            x, y, w, h, area = stats[object_index]

            too_small: bool = area < 400
            not_rectangle: bool = w > h + 10
            if too_small or not_rectangle:
                continue

            roi = original_image[y: y + h, x: x + w]
            
            global i_pedestrian_counter

            cv2.imwrite(f'roi/sample_{i_pedestrian_counter:06d}.png', roi)

            i_pedestrian_counter += 1

            centroid_x, centroid_y = centroids[object_index]

            cv2.rectangle(original_image, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(original_image, str(object_index), (int(centroid_x), int(centroid_y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)

    return original_image

def process_video(file: str) -> None:
    capture = cv2.VideoCapture(file)

    while capture.isOpened():
        has_next_frame, current_frame = capture.read()
        if not has_next_frame:
            break

        new_image = process_frame(current_frame)
        cv2.imshow('IR', new_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()

process_video('vid1_IR.avi')